In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [6]:
spark.version

'3.5.1'

## Практическое задание 2: Работа с продуктами и категориями

In [8]:
## 1. Импорт библиотеки и создание SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

spark = SparkSession.builder.appName('ProductsAndCategories').getOrCreate()

In [9]:
## 2. Создание исходных датафреймов
# Датафрейм с продуктами
products_data = [
    (1, 'Яблоко'),
    (2, 'Молоко'),
    (3, 'Хлеб'),
    (4, 'Молярка'), # продукт без категории
]
products = spark.createDataFrame(products_data, ['product_id', 'product_name'])

In [10]:
# Датафрейм с категориями
categories_data = [
    (100, 'Фрукты'),
    (101, 'Молочные'),
    (102, 'Выпечка'),
]
categories = spark.createDataFrame(categories_data, ['category_id', 'category_name'])

In [11]:
# Связь продукт—категория (многие-ко-многим)
relations_data = [
    (1, 100), # Яблоко — Фрукты
    (2, 101), # Молоко — Молочные
    (3, 102), # Хлеб — Выпечка
    (2, 102), # Молоко — Выпечка (пример для многих категорий)
    # Молярка без категории
]
relations = spark.createDataFrame(relations_data, ['product_id', 'category_id'])

In [12]:
## 3. Метод: получение всех пар и продуктов без категории
from pyspark.sql import DataFrame
from typing import Tuple

def get_product_category_pairs_and_orphans(products: DataFrame, categories: DataFrame, relations: DataFrame) -> Tuple[DataFrame, DataFrame]:
    # Все пары продукт–категория (left join)
    pairs = products.join(relations, on='product_id', how='left') \
                   .join(categories, on='category_id', how='left') \
                   .select('product_name', 'category_name')
    # Все продукты без категории (где category_id == null после join)
    orphans = pairs.filter(col('category_name').isNull()) \
                  .select('product_name')
    return pairs, orphans

In [13]:
## 4. Проверка работы метода (вывод результатов)
pairs_df, orphans_df = get_product_category_pairs_and_orphans(products, categories, relations)
print('Пары продукт–категория:')
pairs_df.show()
print('Продукты без категории:')
orphans_df.show()

Пары продукт–категория:
+------------+-------------+
|product_name|category_name|
+------------+-------------+
|      Яблоко|       Фрукты|
|      Молоко|      Выпечка|
|      Молоко|     Молочные|
|        Хлеб|      Выпечка|
|     Молярка|         NULL|
+------------+-------------+

Продукты без категории:
+------------+
|product_name|
+------------+
|     Молярка|
+------------+



In [14]:
## 5. Тесты (простые asserts)
result_pairs = set([(r['product_name'], r['category_name']) for r in pairs_df.collect()])
result_orphans = set([r['product_name'] for r in orphans_df.collect()])

assert ('Яблоко', 'Фрукты') in result_pairs
assert ('Молоко', 'Молочные') in result_pairs
assert ('Молоко', 'Выпечка') in result_pairs
assert ('Хлеб', 'Выпечка') in result_pairs
assert ('Молярка', None) in result_pairs  # None тут — нет категории
assert 'Молярка' in result_orphans        # Проверяем «сирот»

***

**Подсказка для закрепления:**
- Почему используем 'left join'?
- Почему ищем None (null) в категории?

**Почему используем `left join`?**

`left join` используется потому, что мы хотим получить все продукты, даже если у них нет связанных категорий.  
Если бы использовали обычный (`inner join`), то продукты без категорий просто исчезли бы из результата.

- **`left join`** — сохраняет все строки из левой таблицы (products), даже если для них нет совпадающей строки в правой таблице (relations или categories).
- Именно благодаря этому мы можем увидеть продукты без категорий в итоговом датафрейме.

**Почему ищем `None` (`null`) в категории?**

Когда после объединения (join) для продукта не нашлась соответствующая категория, в поле категории появляется значение `null` (в Spark — `None`).  
То есть, если у продукта нет ни одной связи с категорией, после join в поле `category_name` будет `null`.

- Мы подсвечиваем именно такие случаи — это продукты, не относящиеся ни к одной категории.
- Такой фильтр помогает “вытащить” все продукты-“сироты” для вашего второго условия задачи.

**В итоге:**
- `left join` → чтобы не потерять продукты, даже если нет категории.
- Поиск `None` → чтобы найти “сирот” — продукты без категории.